<a href="https://colab.research.google.com/github/vipin-builds/AI-Engineer-Core-Track/blob/main/notebooks/07_airline_agent_with_voice_and_audio_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create an AI agent that using a sqlite3 database
## 1. Gets the ticket price
## 2. Sets the ticket price

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3
import json
import base64
from io import BytesIO
from PIL import Image

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import os
from google.colab import userdata
from openai import OpenAI
from IPython.display import Markdown, display

# Securely retrieve the key from Colab Secrets
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    client = OpenAI()
    print("✅ OpenAI Client initialized successfully.")
except Exception as e:
    print("❌ Error: Make sure 'OPENAI_API_KEY' is set in the Secrets tab.")

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4.1-mini'

In [ ]:
system_message = "You are an airline assistant"

## And now, setting up the sqlite3 DB

In [ ]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

## Insert a few records.

In [ ]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('INSERT INTO prices VALUES ("india", 2099.99)')
    cursor.execute('INSERT INTO prices VALUES ("london", 1099.99)')
    cursor.execute('INSERT INTO prices VALUES ("australia", 2299.99)')
    cursor.execute('INSERT INTO prices VALUES ("spain", 799.99)')
    cursor.execute('INSERT INTO prices VALUES ("mexico", 299.99)')
    conn.commit()

## Make sure records exist

In [ ]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    # Execute SQL commands
    cursor.execute('SELECT * FROM prices')
    print(cursor.fetchall())


## Add the Get ticket price method

In [ ]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

get_ticket_price("London")

## Add the Set ticket price method

In [ ]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()
    return "True"

set_ticket_price("dubai", 1499.99)

## Image generation

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

## Voice generation

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

## Describe functions

In [ ]:
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "number",
                "description": "The price of the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    },
    "strict": True # Recommended for precise data extraction
}

# And this is included in a list of tools:

tools = [{"type": "function", "function": get_price_function}, {"type": "function", "function": set_price_function}]

## Boilerplate code for handling tool calls.



In [ ]:
def handle_tool_calls_and_return_cities(message):
    responses = []
    cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            get_price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": get_price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            cities.append(city)
            price = arguments.get('price')
            added = set_ticket_price(city, price)
            responses.append({
                "role": "tool",
                "content": added,
                "tool_call_id": tool_call.id
            })
    return responses, cities

## And now, writing a new callback

### We need to check the finish reason to make sure if a tool call is necessary or not.

In [ ]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    cities = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_and_return_cities(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    if cities:
        image = artist(cities[0])

    return history, voice, image

## For Chat history

In [ ]:
def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

## Gradio UI

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True, auth=("vip", "bananas"))